# PROJET 10 DATA ANALYST

# OBJECTIF DE CE NOTEBOOK

Pour l'Organisation Nationale de lutte Contre le Faux-Monnayage (ONCFM), nous devons produire :

- Une analyse descriptive des données, notamment la répartition des dimensions des billets, le nombre de vrais / faux billets, etc.
- Une détection automatisée des faux billets à partir des dimensions de ces derniers. Les méthodes à utiliser sont la régression logistique et k-means avec une matrice de confusion pour évaluer les performances des modèles. Une fois la phase d'entrainement et de test achevée, l'algorithme devra être capable de prédire si un billet est vrai ou faux.

Glossaire :
- diagonal : la diagonale du billet (en mm)
- height_left : la hauteur du billet (mesurée sur le côté gauche, en mm)
- height_right : la hauteur du billet (mesurée sur le côté droit, en mm)
- length : la longueur du billet (en mm)
- margin_low : la marge entre le bord inférieur du billet et l'image de celui-ci (en mm)
- margin_up : la marge entre le bord supérieur du billet et l'image de celui-ci (en mm)

## Etape 1 - Importation des librairies et chargement des fichiers

## 1.1 - Importation des librairies

In [ ]:
#Importation des librairies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score

import scipy.stats as ss

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
#Chargement de la librairie graphique
sns.set()

## 1.2 - Chargement du fichier

In [ ]:
#Importation du fichier population.csv en mettant l'index sur 'Zone'
billet = pd.read_csv('./Data_source/billets.csv', sep=';')

In [ ]:
#Affichage des dimensions et de leurs types
display(billet.info())

Nous pouvons voir que sur les 1500 lignes, la variable 'Height_left' contient des valeurs manquantes.

In [ ]:
#Affichage d'un échantillon
display(billet.sample(5))

## Etape 2 - Analyse exploratoire des données

## 2.1 - Statistiques descriptives

In [ ]:
#Affichage des statistiques descriptives
stats_descr = billet.describe().round(2)
display(stats_descr)

Nous pouvons remarquer 37 valeurs manquantes dans la variable 'margin_low'.

## 2.2 - Analyse univariée

In [ ]:
#Affichage des histogrammes avec la densité de probabilité
for col in stats_descr.columns:
    mu = stats_descr.loc['mean', col]
    sigma = stats_descr.loc['std', col]
    #Règle de Sturges pour déterminer approximativement le nombre optimal de classes
    num_bins = int(np.ceil(np.log2(stats_descr.loc['count', col])) + 1)
    #num_bins = 15
    print(num_bins)

    fig, ax = plt.subplots(figsize=(8, 5))

    #Affichage de l'histogramme
    n, bins, patches = ax.hist(billet[col], num_bins, density=True)

    #Affichage de la densité de probabilité
    y = ((1 / (np.sqrt(2 * np.pi) * sigma)) *
         np.exp(-0.5 * (1 / sigma * (bins - mu))**2))
    ax.plot(bins, y, '--')
    ax.set_xlabel('Valeurs')
    ax.set_ylabel('Densité de probabilité')
    ax.set_title(f"Distribution de '{col}' et densité de probabilité : "
                 fr'$\mu={mu:.2f}$, $\sigma={sigma:.2f}$')

    fig.tight_layout()
    plt.show()

    #Test de Kolmogorov-Smirnov
    print('Si p-value est inférieure à 0.05 alors on rejette H0 = normalité: {}\n\n'.format(ss.kstest(billet[col], 'norm')))

Les densités de probabilité des variables 'margin_low' et 'length' sont éloignées d'une loi normale principalement à cause de la présence des faux billets.

In [ ]:
#Affichage des boxplots
for col in stats_descr.columns:
    sns.boxplot(data=billet[col], orient='h')
    
    plt.title(f"Boxplot de '{col}'")
    plt.show()

Nous ne remarquons pas de valeurs aberrantes dans les variables.

## 2.3 - Analyse bivariée

In [ ]:
#Séparation en deux DataFrames avec et sans NA sur la variable 'margin_low'
billet_isna = billet.loc[billet['margin_low'].isna(), :].copy()
billet_dropna = billet.dropna().copy()

**Matrice de corrélation entre les variables**

In [ ]:
#Matrice des corrélations utilisant le coefficient de corrélation de Pearson
corr_matrix = billet_dropna.iloc[:, 1:].corr(method='pearson', min_periods=20)

#Masque pour la partie triangulaire supérieure de la matrice
mask = np.triu(corr_matrix)

In [ ]:
#Heatmap représentant la matrice des corrélations
plt.figure(figsize=(15,6))
plt.title("Heatmap des coefficients de corrélation de Pearson entre les variables", fontsize=14)

sns.heatmap(corr_matrix, annot=True, vmin=-1, vmax=1, cmap='coolwarm', mask=mask, fmt='.2f')
plt.show()

Nous pouvons voir que les variables 'length' et 'is_genuine' sont très fortement corrélées (0.85). Ce qui serait une piste pour la détection des faux billets à l'aide de la regression logistique...

In [ ]:
#Pairplots pour visualiser les potentielles corrélations
sns.pairplot(billet_dropna, hue='is_genuine', corner=True)
plt.show()

## 2.4 - Imputation des valeurs manquantes

## 2.4.1 - Séparation des données en train et test

In [ ]:
#Sélection des variables explicatives et de la variable à expliquer
y = billet_dropna['margin_low']
X = billet_dropna.drop(['is_genuine', 'margin_low'], axis=1)

In [ ]:
#Séparation des données en train et test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## 2.4.2 - Détermination du nombre optimal de variables explicatives

In [ ]:
#Création de l'espace de cross-validation
folds = KFold(n_splits = 4, shuffle = True, random_state = 0)

#Définition du dictionnaire des variables explicatives à tester
hyper_params = [{'n_features_to_select': list(range(1, 6))}]

#Instanciation du modèle et de la Recursive Feature Elimination (RFE)
lm = LinearRegression()
rfe = RFE(lm)

#Instanciation de la GridSearchCV
model_cv = GridSearchCV(estimator = rfe,
                        param_grid = hyper_params,
                        scoring= 'r2',
                        cv = folds,
                        verbose = 1,
                        return_train_score=True)

#Fit de la GridSearchCV
model_cv.fit(X_train, y_train)

In [ ]:
#Résultats de la GridSearchCV
cv_results = pd.DataFrame(model_cv.cv_results_)

#Sélection des colonnes utiles
cols = [i for i in cv_results.columns if not i.startswith('split') and not i.endswith('time')]
cv_results = cv_results.loc[:, cols]

display(cv_results)

In [ ]:
#Affichage de la contribution des variables explicatives à la performance du modèle
plt.figure(figsize=(15,5))

plt.plot(cv_results["param_n_features_to_select"], cv_results["mean_test_score"])
plt.plot(cv_results["param_n_features_to_select"], cv_results["mean_train_score"])

plt.xlabel('Nombre de variables explicatives')
plt.ylabel('R-squared')
plt.title('Contribution des variables explicatives à la performance du modèle')
plt.legend(['validation score', 'train score'], loc='upper left')
plt.show()

Le nombre optimal de variables explicatives retenu est 2.

## 2.4.3 - Détermination des variables explicatives

In [ ]:
#Choix du nombre optimal de variables explicatives et instanciation du modèle
n_features_optimal = 2
lm = LinearRegression()

# TODO : faire un RFECV sur X_train et y_train ?
#Instanciation de la Recursive Feature Elimination (RFE) et fit
rfe = RFE(lm, n_features_to_select=n_features_optimal)
rfe.fit(X_train, y_train)

In [ ]:
#Affichage de la liste des variables explicatives sélectionnées (True) et non sélectionnées (False) ainsi que leur rang
print(list(zip(X_train.columns,rfe.support_,rfe.ranking_)))

**Les variables explicatives sélectionnées sont 'length' et 'margin_up'.**

## 2.4.4 - Validation de la régression linéaire sur X_test

In [ ]:
#Construction du train set avec les variables explicatives sélectionnées
X_train_2feat = X_train[['length', 'margin_up']].copy()
X_train_2feat = sm.add_constant(X_train_2feat)

#Instanciation et entrainment du modèle
model = sm.OLS(y_train, X_train_2feat).fit()

print(model.summary())

In [ ]:
#Validation du modèle sur le test set
y_pred_test = model.predict(sm.add_constant(X_test[['length', 'margin_up']]))

#Calcul du R2
r2 = r2_score(y_test, y_pred_test)

print(round(r2, 3))

Le R^2 sur le Train Set est comparable à celui sur le Test Set et nous avons une corrélation moyenne (0.45)...

## 2.4.5 - Test des conditions de validité de la régression linéaire


**Vérification des hypothèses :**

- Normalité : Les erreurs résiduelles doivent être distribuées normalement. Cela signifie que les résidus doivent suivre une distribution normale avec une moyenne de zéro.
- Homoscédasticité : L'homoscédasticité signifie que la variance des erreurs résiduelles est constante à tous les niveaux de la variable prédite.
- Multicolinéarité : Cette hypothèse concerne la relation entre les variables prédictives (ou indépendantes) dans notre modèle de régression. Elle stipule qu'il ne devrait pas y avoir de forte corrélation (>0.6) linéaire entre les variables indépendantes.

In [ ]:
#Extraction des valeurs prédites sur le train set
y_pred_train = model.fittedvalues

#Extraction des résidus sur le train set
residues_train = model.resid

**- Normalité des résidus**

In [ ]:
#Définition de la moyenne et de l'écart-type des résidus
mu = residues_train.mean()
sigma = residues_train.std()

#Règle de Sturges pour déterminer approximativement le nombre optimal de classes
num_bins = int(np.ceil(np.log2(residues_train.count())) + 1)

print(num_bins)

fig, ax = plt.subplots(figsize=(8, 5))

#Affichage de l'histogramme
n, bins, patches = ax.hist(residues_train, num_bins, density=True)

#Affichage de la densité de probabilité
y = ((1 / (np.sqrt(2 * np.pi) * sigma)) *
     np.exp(-0.5 * (1 / sigma * (bins - mu))**2))
ax.plot(bins, y, '--')
ax.set_xlabel('Valeurs')
ax.set_ylabel('Densité de probabilité')
ax.set_title(f"Distribution des résidus et densité de probabilité : "
             fr'$\mu={mu:.2f}$, $\sigma={sigma:.2f}$')

fig.tight_layout()
plt.show()

Nous ne sommes pas trop éloigné d'une distribution normale...

Test statistique non-paramétrique de comparaison des distributions (bilatère) : Kolmogorov-Smirnov
- Hypothèse nulle H0 : la densité de probabilité suit une loi normale
- Hypothèse alternative H1 : la densité de probabilité ne suit pas une loi normale
- Seuil alpha de rejet de H0 : 5%

In [ ]:
#Test de Kolmogorov-Smirnov
print('Test de Kolmogorov-Smirnov : {}\n\n'.format(ss.kstest(residues_train, 'norm')))

**- Homoscédasticité des résidus**

In [ ]:
#Calcul des résidus standardisés sur le train set
train_residuals_abs_sqrt=np.sqrt(np.abs(residues_train))

#Affichage des résidus en fonction des valeurs prédites sur le train set
plt.figure(figsize=(8,5))
sns.regplot(x=y_pred_train, y=train_residuals_abs_sqrt,
            scatter=True,
            lowess=True,
            line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})

plt.title('Racine carrée des valeurs absolues des résidus en fonction des valeurs prédites')
plt.ylabel("Standarized residuals")
plt.xlabel("Fitted value")
plt.show()

La variance des résidus est "plutôt" constante à tous les niveaux de la variable prédite.

In [ ]:
#Création d'un DataFrame avec les valeurs prédites et une discrétisation en 3 groupes
y_pred_train_grouped = pd.DataFrame({'y_pred_train': y_pred_train})
y_pred_train_grouped['group'] = pd.qcut(y_pred_train_grouped['y_pred_train'], q=3, labels=False)

#Ajout des résidus standardisés à la DataFrame
y_pred_residues_train_grouped = pd.merge(y_pred_train_grouped, pd.DataFrame({'residues_train': train_residuals_abs_sqrt}), left_index=True, right_index=True, how='inner')

display(y_pred_residues_train_grouped)

Moins de 5% d'Outliers par groupe.

Test statistique paramétrique de comparaison (bilatère) : Levene
- Hypothèse nulle H0 : var0 = var1 = var2
- Hypothèse alternative H1 : au moins une variance diffère
- Seuil alpha de rejet de H0 : 5%

Conditions de validité :
- Si les échantillons à comparer ont une distribution non-normale, choisir le paramètre center='median'

In [ ]:
#Test de Levene
print('Test de Levene :', ss.levene(y_pred_residues_train_grouped.loc[y_pred_residues_train_grouped['group'] == 0,'residues_train'],
                                    y_pred_residues_train_grouped.loc[y_pred_residues_train_grouped['group'] == 1, 'residues_train'],
                                    y_pred_residues_train_grouped.loc[y_pred_residues_train_grouped['group'] == 2, 'residues_train'], center='median'))

**- Multicolinéarité**

In [ ]:
#Calcul des VIF pour les variables explicatives
vif = pd.DataFrame()
vif["Feature"] = X_train_2feat.columns
vif["VIF"] = [variance_inflation_factor(X_train_2feat.values, i) for i in range(X_train_2feat.shape[1])]

display(vif)

Une valeur de VIF entre 1 et 5 est considérée comme acceptable (on ne prend pas en considération 'const' -> Y-intercept). Nous pouvons voir que les variables explicatives sont peu corrélées entre elles...

## 2.4.6 - Prediction des valeurs manquantes

In [ ]:
#Prédiction des valeurs manquantes à l'aide du modèle
billet_isna['margin_low'] = model.predict(sm.add_constant(billet_isna[['length', 'margin_up']])).round(2)

display(billet_isna.sample(10))

## Etape 3 - Sauvegarde des données

In [ ]:
#Concaténation des DataFrames avec et sans NA sur la variable 'margin_low'
billet_compl = pd.concat([billet_isna, billet_dropna], axis=0, ignore_index=True)
billet_compl['is_genuine'].replace([False, True], [0, 1], inplace=True)

display(billet_compl.describe().round(2))
display(billet_compl.sample(10))

In [ ]:
#Sauvegarde du fichier
billet_compl.to_csv('./Data_transformed/billets_compl.csv', sep=';', index=False)